# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv
import matplotlib.pyplot as plt

# Import API key
from api_keys import geoapify_key

In [25]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
print(city_data_df.shape)
city_data_df.head()

(582, 10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,5.75,70,14,6.87,TF,1709933106
1,1,albany,42.6001,-73.9662,12.36,35,75,0.89,US,1709932870
2,2,blackmans bay,-43.0167,147.3167,16.07,90,83,4.23,AU,1709933108
3,3,swakopmund,-22.6833,14.5333,18.47,88,47,1.34,NaN,1709933108
4,4,olonkinbyen,70.9221,-8.7187,0.64,94,93,5.80,SJ,1709933109


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display


# Configure the map plot
city_data_df['Lat'] = city_data_df['Lat'].astype(float)
city_data_df['Lng'] = city_data_df['Lng'].astype(float)
city_data_df['Humidity'] = city_data_df['Humidity'].astype(float)

# Create a geographic plot using hvplot
# The size of the points is scaled by the 'Humidity' column

plot = city_data_df.hvplot.points('Lng', 'Lat', geo=True, c='City', size='Humidity', hover_cols=['City', 'Humidity'], tiles='OSM', frame_width=500, frame_height=400)
# Display the map
#hvplot.show(plot)
plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [24]:
# Narrow down cities that fit criteria and drop any results with null values
#create the variable of ideal_conditions based on 3 factors, guessed from the sample dataset given by the starter codes.
ideal_maxtemp = (city_data_df["Max Temp"]<27) & (city_data_df["Max Temp"]>21)
ideal_cloudiness = (city_data_df["Cloudiness"]==0)
ideal_windspeed = (city_data_df["Wind Speed"]<4.5)
ideal_conditions = ideal_maxtemp & ideal_cloudiness & ideal_windspeed
#create a filter for the ideal condition:

filtered_df = city_data_df[ideal_conditions]
# Drop any rows with null values
# YOUR CODE HERE
filtered_df = filtered_df.dropna()
# Display sample data
# YOUR CODE HERE
print(filtered_df.shape)
filtered_df.head()

(7, 10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
58,58,piti village,13.4626,144.6933,26.49,77.0,0,2.57,GU,1709933136
120,120,la esperanza,14.8667,-91.5667,24.48,60.0,0,0.00,GT,1709933170
167,167,el granada,37.5027,-122.4694,21.08,53.0,0,4.12,US,1709933198
186,186,saipan,15.1355,145.7010,25.39,83.0,0,3.09,MP,1709933209
258,258,macia,-25.0269,33.0989,25.59,84.0,0,3.77,MZ,1709933254


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE

hotel_df=filtered_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['Hotel Name'] = ''
# Display sample data
# YOUR CODE HERE
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
58,piti village,GU,13.4626,144.6933,77.0,
120,la esperanza,GT,14.8667,-91.5667,60.0,
167,el granada,US,37.5027,-122.4694,53.0,
186,saipan,MP,15.1355,145.7010,83.0,
258,macia,MZ,-25.0269,33.0989,84.0,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel

radius = 10000
limit = 1
category = f"accommodation.hotel"

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat_value = row['Lat']
    lng_value = row['Lng']

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    filters = f"circle:{lng_value},{lat_value},{radius}"
    bias = f"proximity:{lng_value},{lat_value}"
    params = {
    "categories":category,
    "filter": filters,
    "bias": bias,
    "limit":limit,
    "apiKey":geoapify_key    
}

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionaty
   
    name_address = requests.get(base_url, params=params)
   
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
piti village - nearest hotel: The Cliff Hotel
la esperanza - nearest hotel: La Esperanza
el granada - nearest hotel: Beach House
saipan - nearest hotel: Chalan Kanoa Beach Hotel
macia - nearest hotel: No hotel found
isla vista - nearest hotel: The Club & Guest House
bandarbeyla - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
58,piti village,GU,13.4626,144.6933,77.0,The Cliff Hotel
120,la esperanza,GT,14.8667,-91.5667,60.0,La Esperanza
167,el granada,US,37.5027,-122.4694,53.0,Beach House
186,saipan,MP,15.1355,145.7010,83.0,Chalan Kanoa Beach Hotel
258,macia,MZ,-25.0269,33.0989,84.0,No hotel found
356,isla vista,US,34.4133,-119.8610,65.0,The Club & Guest House
396,bandarbeyla,SO,9.4942,50.8122,75.0,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot

plot = hotel_df.hvplot.points('Lng', 'Lat', geo=True, c='City', size='Humidity', hover_cols=['City', 'Hotel Name','Country'], tiles='OSM', frame_width=500, frame_height=400)

# Display the map
#hvplot.show(plot)
plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)